In [28]:

import numpy as np
import os
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import time


In [29]:
# A utility function
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

## Creating and Running a Graph


In [30]:
import tensorflow as tf


In [31]:
# Create variables on the default graph
reset_graph()

# Variables and constants

x = tf.Variable(3, name = 'x')
y = tf.Variable(4, name = 'y')
a = tf.constant(2, name = 'a')

In [32]:
# are these nodes on the graph?

print(x.graph is tf.get_default_graph())
print(y.graph is tf.get_default_graph())
print(a.graph is tf.get_default_graph())

True
True
True


In [33]:
# NOTE: we can add nodes to a specific graph, as in this example:
# Here we create a new Graph and temporarily make it the default graph inside a with block, like so:

graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
    
print(x2.graph is graph)
print(x2.graph is tf.get_default_graph())

# To remove duplicate or unwanted nodes:
# tf.reset_default_graph()


True
False


In [34]:
# Variables or constants are not yet initialized:
x,y,a

(<tf.Variable 'x:0' shape=() dtype=int32_ref>,
 <tf.Variable 'y:0' shape=() dtype=int32_ref>,
 <tf.Tensor 'a:0' shape=() dtype=int32>)

In [35]:
# Now define function f(x,y) = x^2 + y + a

f = x*x*y + y + a

In [36]:
# lazy evaluation:

f

<tf.Tensor 'add_1:0' shape=() dtype=int32>

# To evaluate the graph we open a tensorflow session

In [37]:
# A verbose way:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
a_val = a.eval(session=sess) # a constant needs not be initiated
print('a =', a_val)
print('result =', result)
sess.close()

a = 2
result = 42


In [38]:
# Run a session with automatic closing at the end

with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval() # the same as sess.run(f)
    a_val = a.eval()

print('a =', a_val)
print('result =', result)


a = 2
result = 42


# Initialization of all variables at once

In [39]:

# instead of manually running the initializer for every single variable, you can use the
# global_variables_initializer() function. Note that it does not actually perform the initialization
# immediately, but rather creates a node in the graph that will initialize all variables when it is run:

init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run() # actually initialize all variables
    result = f.eval()

print(result)

42


In [40]:
init # this checks what node was created by TensorFlow

<tf.Operation 'init' type=NoOp>

# Lifecycle of a node value

In [41]:

w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

# in this example, code evaluates w and x twice:

with tf.Session() as sess:
    print(y.eval()) # 10
    print(z.eval()) # 15

10
15


# All node values are dropped between runs except variable values
###  variables start their life when they are initialized as a code, and end it when the session is closed.

In [42]:
x

<tf.Tensor 'add_2:0' shape=() dtype=int32>

In [43]:
# in this example, the evaluation of w and x is only done once:

with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val) # 10
    print(z_val) # 15


10
15


# Reverse mode autodiff in Tensorflow

Define a composite function 
\begin{equation*}
f(w)   = exp({w_{20} + w_{21} \cdot exp({w_{10} + w_{11} \cdot exp({w_{00} + w_{01} \cdot x}}})))
\end{equation*}


In [44]:
# Implement in TF

def my_func(w,x):
    f_0 = tf.exp(w[0,0] + w[0,1]*x) # the inner-most function
    f_1 = tf.exp(w[1,0] + w[1,1]*f_0) # the next-level function
    f_2 = tf.exp(w[2,0] + w[2,1]*f_1) # the output function
    return f_2, f_1, f_0


In [45]:

# A fancier implementation using name scopes:

def my_func(w,x):
    with tf.name_scope("f_0_level") as scope_0:
        f_0 = tf.exp(w[0,0] + w[0,1]*x)
    with tf.name_scope("f_1_level") as scope_1:
        f_1 = tf.exp(w[1,0] + w[1,1]*f_0)
    with tf.name_scope("f_2_level") as scope_2:
        f_2 = tf.exp(w[2,0] + w[2,1]*f_1)
    return f_2, f_1, f_0


In [46]:
# w_0 is a point at which we want to compute the function and its derivatives
# w_0 = np.random.rand(3,2)

w_0 = np.vstack((np.zeros(3), np.ones(3))).T

In [47]:
w_0

array([[0., 1.],
       [0., 1.],
       [0., 1.]])

# Compute gradients using TF

In [48]:

reset_graph()

# Variables and their initialization
w = tf.Variable(w_0, name='w', dtype=tf.float32)
x = tf.Variable(1.0, name='x', dtype=tf.float32, trainable=False)

# Define nodes for values of the function at layers
f_2, f_1, f_0 = my_func(w, x)

# Define nodes for the gradients by TensorFlow's autodiff
grads = tf.gradients(f_2, w)
grads


[<tf.Tensor 'gradients/AddN:0' shape=(3, 2) dtype=float32>]

In [51]:

# A node for the initializer
init = tf.global_variables_initializer()

# Run the session
t_0 = time.time()

with tf.Session() as sess:
    sess.run(init)

    # function_vals = sess.run([f_2, f_1, f_0])
    # gradients = sess.run(grads)
    gradients, function_vals = sess.run([grads, [f_2, f_1, f_0]])

print("Computed derivatives in %f3.2 sec" % (time.time() - t_0)) # 0.04 sec if done in two runs
print("Function values = ", function_vals)
print("Gradients = ", gradients)


Computed derivatives in 0.0259453.2 sec
Function values =  [3814273.0, 15.154261, 2.7182817]
Gradients =  [array([[1.5712344e+08, 1.5712344e+08],
       [5.7802488e+07, 1.5712344e+08],
       [3.8142730e+06, 5.7802488e+07]], dtype=float32)]


In [52]:
my_func(w_0, x)

(<tf.Tensor 'f_2_level_1/Exp:0' shape=() dtype=float32>,
 <tf.Tensor 'f_1_level_1/Exp:0' shape=() dtype=float32>,
 <tf.Tensor 'f_0_level_1/Exp:0' shape=() dtype=float32>)

In [36]:
# find code under https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/deepdream/deepdream.ipynb
# functionality for the graph visualization inside Jupyter

from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add()
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = tf.compat.as_bytes("<stripped %d bytes>"%size)
    return strip_def

def show_graph(graph_def, max_const_size=32):

    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
            function load() {{
                document.getElementById("{id}").pbtxt = {data};
            }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
            <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))


# Visualize the tensorflow graph

In [37]:
show_graph(tf.get_default_graph())